In [24]:
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from torch import nn, optim

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = '../DATA'
full_dataset = datasets.ImageFolder(data_dir, transform=transform)

# Splitting the dataset into train and validation sets
train_size = int(0.8 * len(full_dataset))  # 80% of dataset is train
val_size = len(full_dataset) - train_size  # Remaining is validation

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Creating data loaders for train and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

def build_model():
    base_model = models.resnet50(pretrained=True)
    for param in base_model.parameters():
        param.requires_grad = False

    num_features = base_model.fc.in_features
    base_model.fc = nn.Sequential(
        nn.Linear(num_features, 1),
        nn.Sigmoid()
    )
    return base_model.to(device)

In [25]:
model = build_model()

In [28]:
# Set the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [66]:
import torch.nn.functional as F

for epoch in range(32):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs).squeeze()  # Apply sigmoid and squeeze
        loss = criterion(outputs, labels.float())  # Ensure labels are float
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        predicted = (outputs > 0.5).float()  # Outputs are already float, just threshold
        total += labels.size(0)
        correct += predicted.eq(labels.float()).sum().item()  # Ensure labels are float for comparison

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = 100. * correct / total
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

Epoch 1, Loss: 0.6175, Accuracy: 35.27%
Epoch 2, Loss: 0.6196, Accuracy: 35.27%
Epoch 3, Loss: 0.6206, Accuracy: 35.27%
Epoch 4, Loss: 0.6199, Accuracy: 35.27%
Epoch 5, Loss: 0.6187, Accuracy: 35.27%
Epoch 6, Loss: 0.6208, Accuracy: 35.27%
Epoch 7, Loss: 0.6204, Accuracy: 35.27%
Epoch 8, Loss: 0.6209, Accuracy: 35.27%
Epoch 9, Loss: 0.6201, Accuracy: 35.27%
Epoch 10, Loss: 0.6195, Accuracy: 35.27%
Epoch 11, Loss: 0.6194, Accuracy: 35.27%
Epoch 12, Loss: 0.6201, Accuracy: 35.27%
Epoch 13, Loss: 0.6222, Accuracy: 35.27%
Epoch 14, Loss: 0.6226, Accuracy: 35.27%
Epoch 15, Loss: 0.6206, Accuracy: 35.27%
Epoch 16, Loss: 0.6179, Accuracy: 35.27%
Epoch 17, Loss: 0.6205, Accuracy: 35.27%
Epoch 18, Loss: 0.6187, Accuracy: 35.27%
Epoch 19, Loss: 0.6188, Accuracy: 35.27%
Epoch 20, Loss: 0.6203, Accuracy: 35.27%
Epoch 21, Loss: 0.6198, Accuracy: 35.27%
Epoch 22, Loss: 0.6215, Accuracy: 35.27%
Epoch 23, Loss: 0.6172, Accuracy: 35.27%
Epoch 24, Loss: 0.6184, Accuracy: 35.27%
Epoch 25, Loss: 0.6203, A

In [62]:
torch.save(model.state_dict(), '../MODELs/classification_resnet50.pth')

In [63]:
model.load_state_dict(torch.load('../MODELs/classification_resnet50.pth'))

<All keys matched successfully>

In [69]:
# Load the image and apply transformations
from PIL import Image
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

img_path = '../DATA/negative/person_patch_20240427_183537_271742_0.93.png'
image = Image.open(img_path)
input_tensor = transform(image).unsqueeze(0)  # Add a batch dimension

# Move the input tensor to the same device as the model
input_tensor = input_tensor.to(device)

# Make the prediction
with torch.no_grad():
    output = model(input_tensor)

# Assuming the output is a single value for binary classification
prob = torch.sigmoid(output)
threshold = 0.5
predicted_class = 'Positive' if prob >= threshold else 'Negative'
print(f'Predicted Class: {predicted_class}, Probability: {prob.item()}')


Predicted Class: Positive, Probability: 0.5737998485565186
